In [1]:
import numpy as np
from scipy.optimize import linprog

# we try to stick to the notation used in "Computer Solution to the GOPS"

# number of cards used to play 
N = 3

# initial state of the game
init_state = {
    "V": list(range(1,N+1)), # cards in player one's hand
    "Y": list(range(1,N+1)), # cards in player two's hand
    "P": list(range(1,N+1))  # cards in the deck
}

# value of the game for given 
def f(state):
    if not state["P"]:
        return 0
    
    s = 0
    for k in range(0, len(state["P"])):
         s += f_k(state, k)

    return 1/len(state["P"])*s 

def f_k (state, k): 
    return calc_game_value(create_payoff_matrix(state, k))[0]

def new_state (state, i, j, k):
    new_state = {
        "V": state["V"][:i] + state["V"][i+1:],
        "Y": state["Y"][:j] + state["Y"][j+1:],
        "P": state["P"][:k] + state["P"][j+1:]
    }
    return new_state

def create_payoff_matrix(state, k):
    print(state)
    x = lambda i, j: state["P"][k]*np.sign(state["V"][i]-state["Y"][j]) + f(new_state(state, i, j, k))
    n = len(state["P"])

    X = np.empty(shape=(n,n))
    for i in range(0,n):
        for j in range(0,n):
            X[i,j] = x(i,j)
    
    return X

def calc_game_value (X):
    n = X.shape[0]

    # the inequilty constraint matrix is negative
    # because we need to change the inequilty sign
    X = np.append(-X, np.ones(shape=(n,1)), axis=1)

    # inequilty constraint vector
    bub = [0]*n

    # the coefficients of the objective function are negative
    # because scipy.optimize.linprog always solves the minimization problem
    # and we need to maximize
    c = [0]*n 
    c.append(-1)
    c = np.array(c)

    # our only equilty constraint is that the sum of the coefficients is equal to one
    # because we are looking for a probability distribution
    Aeq = [1]*n 
    Aeq.append(0)
    Aeq = np.array(Aeq)
    Aeq = Aeq.reshape(1,n+1)

    beq = 1

    # every coefficient except for the value of the game must be nonnegative
    bounds = [(0,None)]*n 
    bounds.append((None,None))

    result = linprog(c, A_ub=X, b_ub=bub, A_eq=Aeq, b_eq=beq, bounds=bounds)

    return (result.fun, result.x[:-1])

def first(upcard):
    M = create_payoff_matrix(init_state, upcard)
    return calc_game_value(M)[1]

print(first(1))

{'V': [1, 2, 3], 'Y': [1, 2, 3], 'P': [1, 2, 3]}
{'V': [2, 3], 'Y': [2, 3], 'P': [1, 2, 3]}
{'V': [3], 'Y': [3], 'P': [2, 3]}
{'V': [], 'Y': [], 'P': [3]}


IndexError: list index out of range